# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.12,85,100,9.55,TF,1689805107
1,1,olonkinbyen,70.9221,-8.7187,4.90,91,81,4.35,SJ,1689805107
2,2,kralendijk,12.1500,-68.2667,27.51,86,7,10.23,BQ,1689805107
3,3,al khums,32.6486,14.2619,27.29,69,0,1.85,LY,1689805107
4,4,waitangi,-43.9535,-176.5597,13.23,91,84,1.79,NZ,1689805107


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Check column types to ensure humidity is an int
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [4]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    color = 'City',
    alpha = 0.8
)

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# - Temperature in between 23-30 degrees
# - Wind speeds less than 4.5 m/s
# - Cloudiness <= 20
# - Humidity in between 50-70 %

vacation_cities = city_data_df[(city_data_df['Max Temp'] >= 23) & (city_data_df['Max Temp'] <= 30) & 
                               (city_data_df['Wind Speed'] <= 4.5) & (city_data_df['Cloudiness'] <= 20) & 
                               (city_data_df['Humidity'] >= 50) & (city_data_df['Humidity'] <= 70)]

# Drop any rows with null values
vacation_cities_noNull = vacation_cities.dropna()

# Display sample data
vacation_cities_noNull

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,al khums,32.6486,14.2619,27.29,69,0,1.85,LY,1689805107
104,104,jamestown,42.0970,-79.2353,25.85,50,0,4.12,US,1689804939
118,118,manistee,44.2445,-86.3242,25.97,55,20,3.09,US,1689805133
231,231,al matlin,37.2452,10.0500,29.77,65,0,4.48,TN,1689805162
387,387,qobustan,40.0824,49.4120,24.39,62,0,2.18,AZ,1689805197
407,407,ierapetra,35.0100,25.7353,27.61,56,0,3.50,GR,1689805205
474,474,guira de melena,22.7906,-82.5053,29.29,55,13,4.13,CU,1689805227
498,498,pirallahi,40.4723,50.3330,25.08,63,0,3.45,AZ,1689805233
506,506,al bardiyah,31.7561,25.0865,26.59,68,0,2.92,LY,1689805235


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_cities_noNull[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,al khums,LY,32.6486,14.2619,69,
104,jamestown,US,42.0970,-79.2353,50,
118,manistee,US,44.2445,-86.3242,55,
231,al matlin,TN,37.2452,10.0500,65,
387,qobustan,AZ,40.0824,49.4120,62,
407,ierapetra,GR,35.0100,25.7353,56,
474,guira de melena,CU,22.7906,-82.5053,55,
498,pirallahi,AZ,40.4723,50.3330,63,
506,al bardiyah,LY,31.7561,25.0865,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 

# Information for params found here: https://apidocs.geoapify.com/docs/places/#about
params = {
    'categories': 'accommodation.hotel', # https://apidocs.geoapify.com/docs/places/#categories
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search \n---------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df['Lat'][index]
    longitude = hotel_df['Lng'][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{longitude},{latitude},{radius}' # Information on how to set the fitler is taken from this "Filters" table from: https://apidocs.geoapify.com/docs/places/#about 
    params['bias'] = f'proximity:{longitude},{latitude}' # Information on how to set the bias is taken from this "Bias" table from: https://apidocs.geoapify.com/docs/places/#about 
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search 
---------------------
al khums - nearest hotel: فندق الخمس
jamestown - nearest hotel: DoubleTree Jamestown
manistee - nearest hotel: Manistee Inn & Marina
al matlin - nearest hotel: على جانب بيلي
qobustan - nearest hotel: No hotel found
ierapetra - nearest hotel: No hotel found
guira de melena - nearest hotel: No hotel found
pirallahi - nearest hotel: No hotel found
al bardiyah - nearest hotel: فندق البردي


,City,Country,Lat,Lng,Humidity,Hotel Name
3,al khums,LY,32.6486,14.2619,69,فندق الخمس
104,jamestown,US,42.0970,-79.2353,50,DoubleTree Jamestown
118,manistee,US,44.2445,-86.3242,55,Manistee Inn & Marina
231,al matlin,TN,37.2452,10.0500,65,على جانب بيلي
387,qobustan,AZ,40.0824,49.4120,62,No hotel found
407,ierapetra,GR,35.0100,25.7353,56,No hotel found
474,guira de melena,CU,22.7906,-82.5053,55,No hotel found
498,pirallahi,AZ,40.4723,50.3330,63,No hotel found
506,al bardiyah,LY,31.7561,25.0865,68,فندق البردي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    color = "City",
    alpha = 0.8,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)